# Feature Store Quick Start

This notebook gives you a quick overview of how you can intergrate the Feature Store on Hopsworks with Databricks and S3. We'll go over four steps:

1. Generate some sample data and store it on S3
2. Do some feature engineering with Databricks and the data from S3
3. **Save the engineered features to the Feature Store**
4. **Select a group of the features from the Feature Store and create a training dataset of tf records stored on S3**

**This requires a Hopsworks instance that is available from Databricks, see [Databricks Quick Start](https://hopsworks.readthedocs.io/en/latest/getting_started/hopsworksai/guides/databricks_quick_start.html).**

## Imports

We'll use numpy and pandas for data generation, pyspark for feature engineering, tensorflow and keras for model training, and the hops `featurestore` library for interacting with the feature store.

In [3]:
import numpy as np
import random
import pandas as pd
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql import Row
from hops import featurestore
REGION='us-west-2'
SECRETS_STORE='secretsmanager'
PROJECT='demo_featurestore_jim00000'
FEATURE_STORE_HOST='4d20b970-c52a-11ea-a485-f907f59af45e.aws.hopsworks.ai'

--------------------------------------------------------------------------- 
 ModuleNotFoundError Traceback (most recent call last)
 <command-847864287059200> in <module> 
 5 sqlContext = SQLContext ( sc ) 
 6 from pyspark . sql import Row
 ----> 7 from hops import featurestore
 8 REGION = 'us-west-2' 
 9 SECRETS_STORE = 'secretsmanager' 

 ModuleNotFoundError : No module named 'hops'

## Setting up the Feature Store Connector

In [5]:
# setup_databricks only needs to be executed onces per cluster
# Check the documentation for details: https://hopsworks.readthedocs.io/en/latest/getting_started/hopsworksai/guides/databricks_quick_start.html#id8
featurestore.setup_databricks(
  FEATURE_STORE_HOST,
  PROJECT,
  region_name=REGION,
  secrets_store=SECRETS_STORE,
  hostname_verification=True)

In the advanced options of your databricks cluster configuration 
 add the following path to Init Scripts: dbfs:/hops/scripts/initScript.sh

 add the following to the Spark Config:
 spark.hadoop.fs.hopsfs.impl io.hops.hopsfs.client.HopsFileSystem
 spark.hadoop.hops.ipc.server.ssl.enabled true
 spark.hadoop.hops.ssl.hostname.verifier ALLOW_ALL
 spark.hadoop.hops.rpc.socket.factory.class.default io.hops.hadoop.shaded.org.apache.hadoop.net.HopsSSLSocketFactory
 spark.hadoop.client.rpc.ssl.enabled.protocol TLSv1.2
 spark.hadoop.hops.ssl.keystores.passwd.name /dbfs/hops/material_passwd
 spark.hadoop.hops.ssl.keystore.name /dbfs/hops/keyStore.jks
 spark.hadoop.hops.ssl.trustore.name /dbfs/hops/trustStore.jks
 spark.sql.hive.metastore.jars /tmp/apache-hive-bin/lib/*
 spark.hadoop.hive.metastore.uris thrift://ip-10-199-254-9.us-west-2.compute.internal:9083

 Then save and restart the cluster.

## Connecting to the Feature Store

In [7]:
# Connect to the feature store, see https://hopsworks.readthedocs.io/en/latest/getting_started/hopsworksai/guides/databricks_quick_start.html#step-4-3-connecting-to-the-feature-store
featurestore.connect(
  FEATURE_STORE_HOST,
  PROJECT,
  region_name=REGION,
  secrets_store=SECRETS_STORE,
  hostname_verification=True)

## Mounting an S3 bucket to Databricks

In [9]:
# Mount a bucket so that we can simulate a Datalake based on S3
# This requires IAM roles to be set up for Databricks, see https://docs.databricks.com/data/data-sources/aws/amazon-s3.html#access-s3-buckets-directly
AWS_BUCKET_NAME = "jim-oregan" # Ensure to replace with your bucket
MOUNT_NAME = "/mnt/demo_training_datasets"


In [10]:
dbutils.fs.mount("s3a://%s" % AWS_BUCKET_NAME, MOUNT_NAME)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-707875173316342> in <module> 
 ----> 1 dbutils . fs . mount ( "s3a://%s" % AWS_BUCKET_NAME , MOUNT_NAME ) 

 /local_disk0/tmp/1588254532465-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling o292.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/demo_training_datasets; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/demo_training_datasets
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:63)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:465)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/demo_training_datasets
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:224)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:326)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:220)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:79)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:103)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:102)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:102)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:304)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:282)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:51)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:78)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:78)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:47)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:428)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:14)
	at com.databri

## Generate Sample Data

Lets generate two sample datasets and store them on S3:

1. `houses_for_sale_data`:

```bash
+-------+--------+------------------+------------------+------------------+
|area_id|house_id|       house_worth|         house_age|        house_size|
+-------+--------+------------------+------------------+------------------+
|      1|       0| 11678.15482418699|133.88670106643886|366.80067322738535|
|      1|       1| 2290.436167500643|15994.969706808222|195.84014889823976|
|      1|       2| 8380.774578431328|1994.8576926471007|1544.5164614303735|
|      1|       3|11641.224696102923|23104.501275562343|1673.7222604337876|
|      1|       4| 5382.089422436954| 13903.43637058141| 274.2912104765028|
+-------+--------+------------------+------------------+------------------+

 |-- area_id: long (nullable = true)
 |-- house_id: long (nullable = true)
 |-- house_worth: double (nullable = true)
 |-- house_age: double (nullable = true)
 |-- house_size: double (nullable = true)
```
2. `houses_sold_data``
```bash
+-------+-----------------+-----------------+------------------+
|area_id|house_purchase_id|number_of_bidders|   sold_for_amount|
+-------+-----------------+-----------------+------------------+
|      1|                0|                0| 70073.06059070028|
|      1|                1|               15| 146.9198329740602|
|      1|                2|                6|  594.802165433149|
|      1|                3|               10| 77187.84123130841|
|      1|                4|                1|110627.48922722359|
+-------+-----------------+-----------------+------------------+

 |-- area_id: long (nullable = true)
 |-- house_purchase_id: long (nullable = true)
 |-- number_of_bidders: long (nullable = true)
 |-- sold_for_amount: double (nullable = true)
```

We'll use this data for predicting what a house is sold for based on features about the **area** where the house is.

### Generation of `houses_for_sale_data`

In [13]:
area_ids = list(range(1,51))
house_sizes = []
house_worths = []
house_ages = []
house_area_ids = []
for i in area_ids:
    for j in list(range(1,100)):
        house_sizes.append(abs(np.random.normal()*1000)/i)
        house_worths.append(abs(np.random.normal()*10000)/i)
        house_ages.append(abs(np.random.normal()*10000)/i)
        house_area_ids.append(i)
house_ids = list(range(len(house_area_ids)))
houses_for_sale_data  = pd.DataFrame({
        'area_id':house_area_ids,
        'house_id':house_ids,
        'house_worth': house_worths,
        'house_age': house_ages,
        'house_size': house_sizes
    })
houses_for_sale_data_spark_df = sqlContext.createDataFrame(houses_for_sale_data)

In [14]:
houses_for_sale_data_spark_df.show(5)

+-------+--------+------------------+-----------------+------------------+
area_id|house_id| house_worth| house_age| house_size|
+-------+--------+------------------+-----------------+------------------+
 1| 0|22839.193835817092|9517.325107143293|211.87460710460746|
 1| 1| 6666.159123941387|9539.977492332413| 848.3002409923139|
 1| 2| 9364.471016689939|8004.256462264917| 39.70495394765202|
 1| 3| 3228.802593524905|6806.987729189358| 542.2954807823472|
 1| 4|15657.544355162538|5634.052421792608| 538.806720740612|
+-------+--------+------------------+-----------------+------------------+
only showing top 5 rows

In [15]:
houses_for_sale_data_spark_df.printSchema()

root
-- area_id: long (nullable = true)
-- house_id: long (nullable = true)
-- house_worth: double (nullable = true)
-- house_age: double (nullable = true)
-- house_size: double (nullable = true)

In [16]:
houses_for_sale_data_spark_df.write.format("parquet").save("%s/houses_for_sale.parquet" % MOUNT_NAME)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o401.save.
: org.apache.spark.sql.AnalysisException: path dbfs:/mnt/demo_training_datasets/houses_for_sale.parquet already exists.;
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:148)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:126)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:147)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:135)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:188)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:184)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:135)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:118)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:116)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:113)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:242)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:99)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:172)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:306)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:292)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:235)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 AnalysisException Traceback (most recent call last)
 <command-847864287059212> in <module> 
 ----> 1 houses_for_sale_data_spark_df . write . format ( "parquet" ) . save ( "%s/houses_for_sale.parquet" % MOUNT_NAME ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 737 self . _jwrite . save ( ) 
 738 else : 
 --> 739 self . _jwrite . save ( path ) 
 740 
 741 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.7

### Generation of `houses_sold_data`

In [18]:
house_purchased_amounts = []
house_purchases_bidders = []
house_purchases_area_ids = []
for i in area_ids:
    for j in list(range(1,1000)):
        house_purchased_amounts.append(abs(np.random.exponential()*100000)/i)
        house_purchases_bidders.append(int(abs(np.random.exponential()*10)/i))
        house_purchases_area_ids.append(i)
house_purchase_ids = list(range(len(house_purchases_bidders)))
houses_sold_data  = pd.DataFrame({
        'area_id':house_purchases_area_ids,
        'house_purchase_id':house_purchase_ids,
        'number_of_bidders': house_purchases_bidders,
        'sold_for_amount': house_purchased_amounts
    })
houses_sold_data_spark_df = sqlContext.createDataFrame(houses_sold_data)

In [19]:
houses_sold_data_spark_df.show(5)

+-------+-----------------+-----------------+------------------+
area_id|house_purchase_id|number_of_bidders| sold_for_amount|
+-------+-----------------+-----------------+------------------+
 1| 0| 4|195596.37426016538|
 1| 1| 5| 205906.9120954603|
 1| 2| 3| 67978.70131676484|
 1| 3| 1|132724.12607580156|
 1| 4| 13| 73075.43990002344|
+-------+-----------------+-----------------+------------------+
only showing top 5 rows

In [20]:
houses_sold_data_spark_df.printSchema()

root
-- area_id: long (nullable = true)
-- house_purchase_id: long (nullable = true)
-- number_of_bidders: long (nullable = true)
-- sold_for_amount: double (nullable = true)

In [21]:
houses_sold_data_spark_df.write.format("parquet").save("%s/houses_sold.parquet" % MOUNT_NAME)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o486.save.
: org.apache.spark.sql.AnalysisException: path dbfs:/mnt/demo_training_datasets/houses_sold.parquet already exists.;
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:148)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:126)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:147)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:135)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$5.apply(SparkPlan.scala:188)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:184)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:135)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:118)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:116)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter$$anonfun$runCommand$1.apply(DataFrameWriter.scala:710)
	at org.apache.spark.sql.execution.SQLExecution$$anonfun$withCustomExecutionEnv$1.apply(SQLExecution.scala:113)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:242)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:99)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:172)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:710)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:306)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:292)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:235)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)


During handling of the above exception, another exception occurred:

 AnalysisException Traceback (most recent call last)
 <command-847864287059217> in <module> 
 ----> 1 houses_sold_data_spark_df . write . format ( "parquet" ) . save ( "%s/houses_sold.parquet" % MOUNT_NAME ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 737 self . _jwrite . save ( ) 
 738 else : 
 --> 739 self . _jwrite . save ( path ) 
 740 
 741 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4

## Feature Engineering

Lets generate some aggregate features such as sum and averages from our datasets on S3. 

1. `houses_for_sale_features`:

```bash
 |-- area_id: long (nullable = true)
 |-- avg_house_age: double (nullable = true)
 |-- avg_house_size: double (nullable = true)
 |-- avg_house_worth: double (nullable = true)
 |-- sum_house_age: double (nullable = true)
 |-- sum_house_size: double (nullable = true)
 |-- sum_house_worth: double (nullable = true)
```

2. `houses_sold_features`

```bash
 |-- area_id: long (nullable = true)
 |-- avg_num_bidders: double (nullable = true)
 |-- avg_sold_for: double (nullable = true)
 |-- sum_number_of_bidders: long (nullable = true)
 |-- sum_sold_for_amount: double (nullable = true)
```

In [23]:
display(dbutils.fs.ls(MOUNT_NAME))

path,name,size
dbfs:/mnt/demo_training_datasets/houses_for_sale.parquet/,houses_for_sale.parquet/,0
dbfs:/mnt/demo_training_datasets/houses_sold.parquet/,houses_sold.parquet/,0


### Generate Features From `houses_for_sale_data`

In [25]:
houses_for_sale_data_spark_df = spark.read.parquet("%s/houses_for_sale.parquet" % MOUNT_NAME)
sum_houses_for_sale_df = houses_for_sale_data_spark_df.groupBy("area_id").sum()
count_houses_for_sale_df = houses_for_sale_data_spark_df.groupBy("area_id").count()
sum_count_houses_for_sale_df = sum_houses_for_sale_df.join(count_houses_for_sale_df, "area_id")
sum_count_houses_for_sale_df = sum_count_houses_for_sale_df \
    .withColumnRenamed("sum(house_age)", "sum_house_age") \
    .withColumnRenamed("sum(house_worth)", "sum_house_worth") \
    .withColumnRenamed("sum(house_size)", "sum_house_size") \
    .withColumnRenamed("count", "num_rows")
def compute_average_features_house_for_sale(row):
    avg_house_worth = row.sum_house_worth/float(row.num_rows)
    avg_house_size = row.sum_house_size/float(row.num_rows)
    avg_house_age = row.sum_house_age/float(row.num_rows)
    return Row(
        sum_house_worth=row.sum_house_worth, 
        sum_house_age=row.sum_house_age,
        sum_house_size=row.sum_house_size,
        area_id = row.area_id,
        avg_house_worth = avg_house_worth,
        avg_house_size = avg_house_size,
        avg_house_age = avg_house_age
       )
houses_for_sale_features_df = sum_count_houses_for_sale_df.rdd.map(
    lambda row: compute_average_features_house_for_sale(row)
).toDF()

In [26]:
houses_for_sale_features_df.printSchema()

root
-- area_id: long (nullable = true)
-- avg_house_age: double (nullable = true)
-- avg_house_size: double (nullable = true)
-- avg_house_worth: double (nullable = true)
-- sum_house_age: double (nullable = true)
-- sum_house_size: double (nullable = true)
-- sum_house_worth: double (nullable = true)

### Generate Features from `houses_sold_data`

In [28]:
houses_sold_data_spark_df = spark.read.parquet("%s/houses_sold.parquet" % MOUNT_NAME)
sum_houses_sold_df = houses_sold_data_spark_df.groupBy("area_id").sum()
count_houses_sold_df = houses_sold_data_spark_df.groupBy("area_id").count()
sum_count_houses_sold_df = sum_houses_sold_df.join(count_houses_sold_df, "area_id")
sum_count_houses_sold_df = sum_count_houses_sold_df \
    .withColumnRenamed("sum(number_of_bidders)", "sum_number_of_bidders") \
    .withColumnRenamed("sum(sold_for_amount)", "sum_sold_for_amount") \
    .withColumnRenamed("count", "num_rows")
def compute_average_features_houses_sold(row):
    avg_num_bidders = row.sum_number_of_bidders/float(row.num_rows)
    avg_sold_for = row.sum_sold_for_amount/float(row.num_rows)
    return Row(
        sum_number_of_bidders=row.sum_number_of_bidders, 
        sum_sold_for_amount=row.sum_sold_for_amount,
        area_id = row.area_id,
        avg_num_bidders = avg_num_bidders,
        avg_sold_for = avg_sold_for
       )
houses_sold_features_df = sum_count_houses_sold_df.rdd.map(
    lambda row: compute_average_features_houses_sold(row)
).toDF()

In [29]:
houses_sold_features_df.printSchema()

root
-- area_id: long (nullable = true)
-- avg_num_bidders: double (nullable = true)
-- avg_sold_for: double (nullable = true)
-- sum_number_of_bidders: long (nullable = true)
-- sum_sold_for_amount: double (nullable = true)

## Save Features to the Feature Store

The Featue store has an abstraction of a **feature group** which is a set of features that naturally belong together that typically are computed using the same feature engineering job and the same raw dataset. 

Lets create two feature groups:

1. `houses_for_sale_featuregroup`

2. `houses_sold_featuregroup`

In [31]:
featurestore.create_featuregroup(
    houses_for_sale_features_df,
    "houses_for_sale_featuregroup",
    description="aggregate features of houses for sale per area",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False
)

--------------------------------------------------------------------------- 
 RestAPIError Traceback (most recent call last)
 /databricks/python/lib/python3.7/site-packages/hops/featurestore.py in create_featuregroup (df, featuregroup, primary_key, description, featurestore, featuregroup_version, jobs, descriptive_statistics, feature_correlation, feature_histograms, cluster_analysis, stat_columns, num_bins, corr_method, num_clusters, partition_by, online, online_types, offline) 
 637 corr_method = corr_method , num_clusters = num_clusters , partition_by = partition_by , 
 --> 638 online=online, online_types=online_types, offline=offline)
 639 # If it fails, update cache and retry 

 /databricks/python/lib/python3.7/site-packages/hops/featurestore_impl/core.py in _do_create_featuregroup (df, featurestore_metadata, featuregroup, primary_key, description, featurestore, featuregroup_version, jobs, descriptive_statistics, feature_correlation, feature_histograms, cluster_analysis, stat_columns, num_bins, corr_method, num_clusters, partition_by, online, online_types, offline) 
 1851 num_bins , num_clusters , corr_method , featuregroup_type , featuregroup_type_dto , 
 -> 1852 None, None, online)
 1853 fs_utils . _log ( "Registering feature metadata... [COMPLETE]" ) 

 /databricks/python/lib/python3.7/site-packages/hops/featurestore_impl/rest/rest_rpc.py in _create_featuregroup_rest (featuregroup, featurestore_id, description, featuregroup_version, jobs, features_schema, feature_corr_data, featuregroup_desc_stats_data, features_histogram_data, cluster_analysis_data, feature_corr_enabled, featuregroup_desc_stats_enabled, features_histogram_enabled, cluster_analysis_enabled, stat_columns, num_bins, num_clusters, corr_method, featuregroup_type, featuregroup_dto_type, sql_query, jdbc_connector_id, online_fg) 
 293 "HTTP code: {}, HTTP reason: {}, error code: {}, error msg: {}, user msg: {}".format(
 --> 294 resource_url, response.status_code, response.reason, error_code, error_msg, user_msg))
 295 

 RestAPIError : Could not create feature group (url: /hopsworks-api/api/project/119/featurestores/67/featuregroups), server response: 
 HTTP code: 400, HTTP reason: Bad Request, error code: 270089, error msg: The feature group you are trying to create does already exist., user msg: project: demo_featurestore_jim00000, featurestoreId: 67

During handling of the above exception, another exception occurred:

 RestAPIError Traceback (most recent call last)
 <command-847864287059227> in <module> 
 6 feature_correlation = False , 
 7 feature_histograms = False , 
 ----> 8 cluster_analysis = False 
 9 )

 /databricks/python/lib/python3.7/site-packages/hops/featurestore.py in create_featuregroup (df, featuregroup, primary_key, description, featurestore, featuregroup_version, jobs, descriptive_statistics, feature_correlation, feature_histograms, cluster_analysis, stat_columns, num_bins, corr_method, num_clusters, partition_by, online, online_types, offline) 
 646 cluster_analysis = cluster_analysis , stat_columns = stat_columns , num_bins = num_bins , 
 647 corr_method = corr_method , num_clusters = num_clusters , partition_by = partition_by , 
 --> 648 online=online, online_types=online_types, offline=offline)
 649 
 650 # update metadata cache since we created a new feature group and added new metadata 

 /databricks/python/lib/python3.7/site-packages/hops/featurestore_impl/core.py in _do_create_featuregroup (df, featurestore_metadata, featuregroup, primary_key, description, featurestore, featuregroup_version, jobs, descriptive_statistics, feature_correlation, feature_histograms, cluster_analysis, stat_columns, num_bins, corr_method, num_clusters, partition_by, online, online_types, offline) 
 1850 descriptive_statistics , feature_histograms , cluster_analysis , stat_columns , 
 1851 num_bins , num_clusters , corr_method , featuregroup_type , featuregroup_type_dto , 
 -> 1852 None, None, online)
 1853 fs_utils . _log ( "Registering feature metadata...

In [32]:
featurestore.create_featuregroup(
    houses_sold_features_df,
    "houses_sold_featuregroup",
    description="aggregate features of sold houses per area",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False
)

Registering feature metadata...
Registering feature metadata... [COMPLETE]
Writing feature data to offline feature group (Hive)...
Running sql: use demo_featurestore_jim00000_featurestore against offline feature store
Writing feature data to offline feature group (Hive)... [COMPLETE]
Feature group created successfully

## Create a Training Dataset

The feature store has an abstraction of a **training dataset**, which is a dataset with a set of features (potentially from many different feature groups) and labels (in case of supervised learning). 

Let's create a training dataset called *predict_house_sold_for_dataset* using the following features:

- avg_house_age
- avg_house_size
- avg_house_worth
- avg_num_bidders

and the target variable is:

- avg_sold_for

In [34]:
features_df = featurestore.get_features(["avg_house_age", "avg_house_size", 
                                         "avg_house_worth", "avg_num_bidders", 
                                         "avg_sold_for"])

featurestore.create_training_dataset(
    features_df, "predict_house_sold_for_dataset_two",
    data_format="tfrecords",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False,
    sink='jim-oregan' # See https://hopsworks.readthedocs.io/en/latest/featurestore/guides/featurestore.html#configuring-storage-connectors-for-the-feature-store
)

Running sql: use demo_featurestore_jim00000_featurestore against offline feature store
Logical query plan for getting 5 features from the featurestore created successfully
SQL string for the query created successfully
Running sql: SELECT avg_sold_for, avg_house_age, avg_house_size, avg_num_bidders, avg_house_worth FROM houses_sold_featuregroup_1 JOIN houses_for_sale_featuregroup_1 ON houses_sold_featuregroup_1.`area_id`=houses_for_sale_featuregroup_1.`area_id` against offline feature store
write feature frame, write_mode: overwrite
Training Dataset created successfully

In [35]:
features_df.show(5)

+------------------+------------------+------------------+--------------------+------------------+
 avg_sold_for| avg_house_age| avg_house_size| avg_num_bidders| avg_house_worth|
+------------------+------------------+------------------+--------------------+------------------+
 3250.164812645495| 261.2603487984262|23.953524668767752|0.043043043043043044|259.63553145772033|
 2377.364099766353|184.93581102333127|16.744133746387067|0.014014014014014014| 188.3377117070782|
 3283.470920463867|245.18125243592596|28.894407116303206| 0.05305305305305305| 286.5947879163963|
2463.8355210458567|167.80308871239976| 18.31532898187385|0.018018018018018018|191.01302219730556|
 2123.221433295513|172.61531843664673|15.745658708242688|0.011011011011011011|164.82636105543025|
+------------------+------------------+------------------+--------------------+------------------+
only showing top 5 rows

In [36]:
features_df = featurestore.get_features(
    ["team_budget", "average_attendance", "average_player_age",
    "team_position", "sum_attendance", 
     "average_player_rating", "average_player_worth", "sum_player_age",
     "sum_player_rating", "sum_player_worth", "sum_position", 
     "average_position"
    ]
)

Running sql: use demo_featurestore_jim00000_featurestore against offline feature store
Logical query plan for getting 12 features from the featurestore created successfully
SQL string for the query created successfully
Running sql: SELECT team_budget, average_position, sum_player_rating, average_attendance, average_player_worth, sum_player_worth, sum_position, sum_attendance, average_player_rating, team_position, sum_player_age, average_player_age FROM teams_features_1 JOIN season_scores_features_1 JOIN players_features_1 JOIN attendances_features_1 ON teams_features_1.`team_id`=season_scores_features_1.`team_id` AND teams_features_1.`team_id`=players_features_1.`team_id` AND teams_features_1.`team_id`=attendances_features_1.`team_id` against offline feature store

In [37]:
display(dbutils.fs.ls(MOUNT_NAME))

path,name,size
dbfs:/mnt/demo_training_datasets/TRAINING_DATASETS/,TRAINING_DATASETS/,0
dbfs:/mnt/demo_training_datasets/houses_sold.parquet/,houses_sold.parquet/,0
